In [3]:
import keras
from keras.datasets import mnist
import pickle, numpy
from sklearn.model_selection import train_test_split

## Load the data from Keras dataset<br>
Returns two tuples:<br>
X, X_test: uint8 array of RGB image data with shape (num_samples, 3, 32, 32).<br>
y, y_test: uint8 array of category labels (integers in range 0-9) with shape (num_samples,).

In [4]:
# the data, split between train and test sets
(X,y), (X_test, y_test) = mnist.load_data()

In [5]:
# split training set into training and validation sets:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

## Write data to project filesystem

In [6]:
with open('mnist-tf-train.pkl', 'wb') as f:
    pickle.dump((X_train, y_train), f, protocol=pickle.HIGHEST_PROTOCOL)

with open('mnist-tf-valid.pkl', 'wb') as f:
    pickle.dump((X_valid, y_valid), f, protocol=pickle.HIGHEST_PROTOCOL)

with open('mnist-tf-test.pkl', 'wb') as f:
    pickle.dump((X_test, y_test), f, protocol=pickle.HIGHEST_PROTOCOL)

In [7]:
import ibm_boto3
from ibm_botocore.client import Config

## Insert Cloud Object Storage credentials 

1. Select the Data icon (upper right, icon with 1 and 0s)
2. Select Connections
3. Select the cell below and then select <b>Insert to code</b> for <b>Connection to project COS</b> to insert credentials
4. Rename the credentials (which are usually called credentials_1) to cos_credentials.   YOU WILL GET ERRORS IF YOU DO NOT DO THIS!

In [8]:

# @hidden_cell
credentials_1 = {
  'secret_key':'0e0600e5f6bb917f56f6f83b3e269efb17879e785a52e375',
  'iam_url':'https://iam.ng.bluemix.net/oidc/token',
  'api_key':'g13BP95BY-k0ZAr_5mDUxETkQsj0Xg68NiIDw6fLx_hH',
  'resource_instance_id':'crn:v1:bluemix:public:cloud-object-storage:global:a/affc9aad9ed36815e657b2cc4d2616bd:60e98ac9-8749-4d82-b544-b6fcb4933c7f::',
  'access_key':'6c3128b29392463bb549904b8ed7484c',
  'url':'https://s3-api.us-geo.objectstorage.service.networklayer.com'
}


In [10]:
service_endpoint = credentials_1['url']  

In [11]:
auth_endpoint = credentials_1['iam_url']  

In [13]:
cos = ibm_boto3.client('s3',
                      ibm_api_key_id=credentials_1['api_key'],
                      ibm_service_instance_id=credentials_1['resource_instance_id'],
                      ibm_auth_endpoint=auth_endpoint,
                      config=Config(signature_version='oauth'),
                      endpoint_url=service_endpoint)

## Create bucket name -- I usually append my name to the end (i.e. cifar10-tutorials-joel)

In [14]:
bucket = "mnist-demo"

## This call may fail if bucket has already been created -- that's fine, simply continue executing the next cells

In [15]:
files = ['mnist-tf-train.pkl',
         'mnist-tf-valid.pkl',
         'mnist-tf-test.pkl']

In [16]:
for file in files:
    print('Uploading data {}...', format(file))
    cos.upload_file(Filename=file, Bucket=bucket, Key=file)

Uploading data {}... mnist-tf-train.pkl
Uploading data {}... mnist-tf-valid.pkl
Uploading data {}... mnist-tf-test.pkl
